In [75]:
import json
import pickle
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
from collections import defaultdict

In [80]:
OUTPUT_PICKLE_PATH = 'output_with_explanations.pkl'
df = pd.read_pickle(OUTPUT_PICKLE_PATH)

In [83]:
df[df['explanation'].isna()]

,method_change_id,file_change_id,name,programming_language,start_line,end_line,nloc,before_change,cwe_id,label,code,explanation
6315,203290764683358,243574523519824,e.prototype.toTraceparent,JavaScript,2,2,1.0,False,CWE-79,non_vuln,"(()=>{var e={8554:(e,t,n)=>{""use strict"";n.d(t...",NaN
6316,40589906276934,258756040035095,v.resizeCell,JavaScript,10819,10819,1.0,False,CWE-79,non_vuln,"x,y,I){q.apply(this,arguments);g()};v.resizeCe...",NaN
6317,212430453477103,258756040035095,x.getModel,JavaScript,11661,11671,5.0,False,CWE-79,non_vuln,"""..."");m.value=c;x.updateCellSize(m);if(0<this...",NaN
6318,23769780375822,225455639466281,Editor.MathJaxRender,JavaScript,11518,11518,1.0,False,CWE-79,non_vuln,"function(P){""undefined""!==typeof MathJax&&""fun...",NaN
6319,238481029561351,88580200346555,parseFloat,JavaScript,1112,1114,3.0,False,CWE-79,non_vuln,"(sa=!0,oa++);L(qa,x,A,C,(W.OL?W.LiIndex+"". "":""...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9018,108591795278411,80362477205770,c.amount,TypeScript,106,114,9.0,False,CWE-79,non_vuln,"? `${c.amount(a.value, a.curre...",NaN
9019,176058316263243,42955047710529,validateUrl,TypeScript,12,29,15.0,False,CWE-79,non_vuln,"): boolean => {\n const { allowedSchemes, isU...",NaN
9020,147388470633125,210385506318845,update,TypeScript,44,44,1.0,False,CWE-79,non_vuln,): { destroy: () => void; update: (t: () => To...,NaN
9021,239788865929483,255575629232229,metaConfigExtended,TypeScript,182,188,7.0,False,CWE-89,non_vuln,async metaConfigExtended(options) {\n con...,NaN


In [76]:
# result_file_path = "../result/file_level_base/04-11_21-54_zero_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/file_level_base/04-11_21-54_cot_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/file_level_base/04-11_21-54_fs_prompt/final_results_phi4:14b.json"

# result_file_path = "../result/func_level_base/04-11_17-55_zero_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/func_level_base/04-10_09-29_cot_prompt/final_results_phi4:14b.json"
# result_file_path = "../result/func_level_base/04-10_09-29_fs_prompt/final_results_phi4:14b.json"

# result_file_path = "../result/FT_file_result/04-18_08-37_zero_prompt/final_results_Phi4_FT_3ep_file.json"
# result_file_path = "../result/FT_file_result/04-18_08-37_cot_prompt/final_results_Phi4_FT_3ep_file.json"
result_file_path = "../result/FT_file_result/04-18_08-37_fs_prompt/final_results_Phi4_FT_3ep_file.json"

# result_file_path = "../result/FT_func_result/04-15_12-47_zero_prompt/final_results_Phi4_FT_3ep.json"
# result_file_path = "../result/FT_func_result/04-15_12-47_cot_prompt/final_results_Phi4_FT_3ep.json"
# result_file_path = "../result/FT_func_result/04-15_12-47_fs_prompt/final_results_Phi4_FT_3ep.json"

# Load the JSON file
with open(result_file_path, 'r') as file:
    results = json.load(file)

print(len(results))

494


# Result Analysis

In [77]:
from random import randint 
print(json.dumps(results[randint(0, len(results))], indent=4))

{
    "index": 139,
    "file_change_id": "16131904614361",
    "lang": "PHP",
    "vuln_type": "CWE-79",
    "result": {
        "reasoning": "The code snippet provided is part of a PHP application that uses the PhpMyAdmin library to lint SQL queries. The SQL query received from the user via POST request is not executed but parsed and returned as a JSON response. Since the query is never executed, there is no risk of SQL injection (CWE-89). Additionally, because the output is encoded in JSON format and any HTML fragments are handled by the client-side application, there is no risk of Cross-Site Scripting (XSS) vulnerabilities (CWE-79). The code also does not involve file uploads or CSRF tokens, so CWE-434 and CWE-352 do not apply. Furthermore, there is no evidence of improper input validation that could lead to business logic attacks (CWE-20).",
        "cwe_id": "None"
    },
    "model": "hf.co/Kei5uke/Phi4_trained_with_file:latest",
    "version": 1,
    "error": false,
    "time":

In [78]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def calculate_weighted_metrics(true_label, pred_label):
    """
    Calculate weighted classification metrics (Accuracy, Recall, Precision, F1).
    
    Args:
        true_label (array-like): Ground truth labels.
        pred_label (array-like): Predicted labels.
        
    Returns:
        dict: Weighted metrics (Accuracy, Recall, Precision, F1).
    """
    # print(np.unique(true_label))
    # print(np.unique(pred_label))
    cwe_classes = np.unique(true_label)
    cwe_cm = confusion_matrix(true_label, pred_label, labels=cwe_classes)
    # Weighted metrics (adjusts for class imbalance by weighting classes by their support)
    acc = accuracy_score(true_label, pred_label)
    recall = recall_score(true_label, pred_label, average='weighted')  # Weighted Recall (Sensitivity)
    precision = precision_score(true_label, pred_label, average='weighted')  # Weighted Precision
    f1 = f1_score(true_label, pred_label, average='weighted')  # Weighted F1-score

    # Calculate global FNR and FPR
    sum_TP = cwe_cm.diagonal().sum()
    total_samples = cwe_cm.sum()
    sum_FN = total_samples - sum_TP
    global_FNR = sum_FN / total_samples

    number_of_classes = len(cwe_classes)
    if number_of_classes > 1:
        global_FPR = sum_FN / ((number_of_classes - 1) * total_samples)
    else:
        global_FPR = 0.0  # Handle case with only one class
    
    return {
        "Weighted Accuracy": acc,
        "Weighted Recall": recall,
        "Weighted Precision": precision,
        "Weighted F1-score": f1,
        "Weighted FNR": global_FNR,
        "Weighted FPR": global_FPR
    }

In [79]:
chosen_cwes = ['79', '89', '434', '352']
langs = ['PHP', 'JavaScript', 'Java', 'TypeScript', 'Ruby', 'Python']

def judge_cwe(cwe):
    for chosen_cwe in chosen_cwes:
        if chosen_cwe in cwe:
            return 'CWE-'+ chosen_cwe
    return 'None'

for lang in langs:
  true_label = []
  pred_label = []
  for result in results:
    if result['lang'] == lang:
      true = result['vuln_type']

      if true == 'non_vuln': true = 'None'

      if(result['result'].get('cwe_id') != None):
        pred = judge_cwe(result['result']['cwe_id'])
        true_label.append(true)
        pred_label.append(pred)

    
  metrics = calculate_weighted_metrics(true_label, pred_label)
  print(f"Language: {lang}")
  for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")


Language: PHP
Weighted Accuracy: 0.3512
Weighted Recall: 0.3512
Weighted Precision: 0.3694
Weighted F1-score: 0.3534
Weighted FNR: 0.6488
Weighted FPR: 0.1622
Language: JavaScript
Weighted Accuracy: 0.5000
Weighted Recall: 0.5000
Weighted Precision: 0.5170
Weighted F1-score: 0.4940
Weighted FNR: 0.5000
Weighted FPR: 0.1667
Language: Java
Weighted Accuracy: 0.4583
Weighted Recall: 0.4583
Weighted Precision: 0.4113
Weighted F1-score: 0.4305
Weighted FNR: 0.5417
Weighted FPR: 0.1806
Language: TypeScript
Weighted Accuracy: 0.5357
Weighted Recall: 0.5357
Weighted Precision: 0.5714
Weighted F1-score: 0.5434
Weighted FNR: 0.4643
Weighted FPR: 0.1548
Language: Ruby
Weighted Accuracy: 0.5000
Weighted Recall: 0.5000
Weighted Precision: 0.4083
Weighted F1-score: 0.4491
Weighted FNR: 0.5000
Weighted FPR: 0.1250
Language: Python
Weighted Accuracy: 0.5455
Weighted Recall: 0.5455
Weighted Precision: 0.6076
Weighted F1-score: 0.5269
Weighted FNR: 0.4545
Weighted FPR: 0.1515


/Users/keisuke_konno/code/python3/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/keisuke_konno/code/python3/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
